In [ ]:
!apt-get update
!apt-get install openjdk-8-jdk -y

Get:1 https://cloud.r-project.org/bin/linux/ubuntu jammy-cran40/ InRelease [3,632 B]
Hit:2 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu2204/x86_64  InRelease
Get:3 https://r2u.stat.illinois.edu/ubuntu jammy InRelease [6,555 B]
Get:4 http://security.ubuntu.com/ubuntu jammy-security InRelease [129 kB]
Hit:5 http://archive.ubuntu.com/ubuntu jammy InRelease
Get:6 http://archive.ubuntu.com/ubuntu jammy-updates InRelease [128 kB]
Hit:7 https://ppa.launchpadcontent.net/deadsnakes/ppa/ubuntu jammy InRelease
Get:8 https://r2u.stat.illinois.edu/ubuntu jammy/main amd64 Packages [2,677 kB]
Hit:9 https://ppa.launchpadcontent.net/graphics-drivers/ppa/ubuntu jammy InRelease
Hit:10 https://ppa.launchpadcontent.net/ubuntugis/ppa/ubuntu jammy InRelease
Get:11 https://r2u.stat.illinois.edu/ubuntu jammy/main all Packages [8,762 kB]
Get:12 http://archive.ubuntu.com/ubuntu jammy-backports InRelease [127 kB]
Get:13 http://security.ubuntu.com/ubuntu jammy-security/universe amd64 Packages [1

In [ ]:
# Step 1: Download Spark
!wget https://archive.apache.org/dist/spark/spark-3.1.2/spark-3.1.2-bin-hadoop3.2.tgz

--2025-03-21 06:10:09--  https://archive.apache.org/dist/spark/spark-3.1.2/spark-3.1.2-bin-hadoop3.2.tgz
Resolving archive.apache.org (archive.apache.org)... 65.108.204.189, 2a01:4f9:1a:a084::2
Connecting to archive.apache.org (archive.apache.org)|65.108.204.189|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 228834641 (218M) [application/x-gzip]
Saving to: ‘spark-3.1.2-bin-hadoop3.2.tgz’

spark-3.1.2-bin-had 100%[===================>] 218.23M  5.14MB/s    in 30s     

2025-03-21 06:10:40 (7.32 MB/s) - ‘spark-3.1.2-bin-hadoop3.2.tgz’ saved [228834641/228834641]



In [ ]:
!pip install gradio

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 46.2/46.2 MB 18.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 322.2/322.2 kB 19.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 94.9/94.9 kB 5.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 11.3/11.3 MB 36.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 72.0/72.0 kB 4.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 62.3/62.3 kB 4.6 MB/s eta 0:00:00


In [ ]:
pip install pandas gradio pyspark matplotlib


In [ ]:
# Step 2: Extract Spark
!ls -l
!tar xf spark-3.1.2-bin-hadoop3.2.tgz
!ls -l

# Step 3: Set Environment Variables
import os
os.environ["JAVA_HOME"] = "/usr/lib/jvm/java-8-openjdk-amd64"
os.environ["SPARK_HOME"] = "/content/spark-3.1.2-bin-hadoop3.2"

# Step 4: Initialize findspark
!pip install -q findspark
import findspark
findspark.init()
print("SPARK_HOME is set to:", os.environ["SPARK_HOME"])

# Step 5: Verify Spark Installation
!pip install -q pyspark
from pyspark.sql import SparkSession
spark = SparkSession.builder.master("local[*]").appName("TestSpark").getOrCreate()
print("Spark session created successfully!")
spark

total 223480
drwxr-xr-x 1 root root      4096 Mar 19 13:34 sample_data
-rw-r--r-- 1 root root 228834641 May 24  2021 spark-3.1.2-bin-hadoop3.2.tgz
total 223484
drwxr-xr-x  1 root root      4096 Mar 19 13:34 sample_data
drwxr-xr-x 13 1000 1000      4096 May 24  2021 spark-3.1.2-bin-hadoop3.2
-rw-r--r--  1 root root 228834641 May 24  2021 spark-3.1.2-bin-hadoop3.2.tgz
SPARK_HOME is set to: /content/spark-3.1.2-bin-hadoop3.2
Spark session created successfully!


In [ ]:
import os
import json
import pandas as pd
import re
import gradio as gr
import matplotlib.pyplot as plt
from pyspark.sql import SparkSession
from pyspark.sql.functions import udf
from pyspark.sql.types import ArrayType, StringType
import subprocess

# -------------------------------
# Inconsistency Detection Functions
# -------------------------------

def detect_formatting_issues(text):
    """
    Checks whether the text:
      - Starts with an uppercase letter.
      - Ends with proper punctuation (., !, or ?).
    Returns a list of formatting issues.
    """
    issues = []
    if not text:
        issues.append("Empty text")
        return issues
    if not text[0].isupper():
        issues.append("Does not start with an uppercase letter")
    if text[-1] not in ['.', '!', '?']:
        issues.append("Does not end with proper punctuation")
    return issues

def detect_conflicting_information(text):
    """
    Checks for pairs of contradictory keywords in the text.
    Returns a list of detected conflicts.
    """
    conflict_pairs = [
        ('increase', 'decrease'),
        ('up', 'down'),
        ('true', 'false'),
        ('positive', 'negative'),
        ('win', 'loss'),
        ('hot', 'cold')
    ]
    conflicts = []
    text_lower = text.lower()
    for w1, w2 in conflict_pairs:
        if w1 in text_lower and w2 in text_lower:
            conflicts.append(f"Conflict: {w1} vs {w2}")
    return conflicts

def detect_data_anomalies(text):
    """
    Detects data anomalies such as:
      - Empty text.
      - Text that is extremely short.
      - Excessive punctuation (three or more punctuation marks in a row).
      - A high ratio of non-alphanumeric to alphanumeric characters.
    Returns a list of anomaly issues.
    """
    anomalies = []
    if not text or len(text.strip()) == 0:
        anomalies.append("Empty text anomaly")
    if len(text) < 5:
        anomalies.append("Text too short")
    if re.search(r'[\!\?\.]{3,}', text):
        anomalies.append("Excessive punctuation")
    alpha_numeric = re.findall(r'[a-zA-Z0-9]', text)
    non_alphanumeric = re.findall(r'[^a-zA-Z0-9\s]', text)
    if alpha_numeric and (len(non_alphanumeric) / len(alpha_numeric)) > 0.5:
        anomalies.append("High non-alphanumeric ratio")
    return anomalies

def detect_duplicates(df, text_column='text_column'):
    """
    Returns a DataFrame with rows that have duplicate values in the specified text column.
    """
    duplicates = df[df.duplicated(subset=[text_column], keep=False)]
    return duplicates

def correct_text(text):
    """
    Corrects formatting issues:
      - Capitalizes the first character if necessary.
      - Ensures the text ends with a punctuation mark (adds a period if missing).
    Returns the corrected text.
    """
    if not text:
        return text
    if not text[0].isupper():
        text = text[0].upper() + text[1:]
    if text[-1] not in ['.', '!', '?']:
        text = text + '.'
    return text

# -------------------------------
# Hadoop Integration Function
# -------------------------------
# Set this flag to True if Hadoop is available; otherwise, leave it False.
ENABLE_HADOOP = False

def upload_to_hdfs(local_file, hdfs_path):
    """
    Uploads a local file to HDFS using a shell command.
    Requires Hadoop to be installed and HDFS running.
    """
    try:
        result = subprocess.run(
            ["hdfs", "dfs", "-put", "-f", local_file, hdfs_path],
            check=True,
            stdout=subprocess.PIPE,
            stderr=subprocess.PIPE
        )
        return f"File uploaded to HDFS at: {hdfs_path}"
    except subprocess.CalledProcessError as e:
        return f"Failed to upload to HDFS: {e.stderr.decode('utf-8')}"

# -------------------------------
# Gradio Interface Processing Function with Spark & Optional Hadoop
# -------------------------------

def process_file(file_obj):
    if file_obj is None:
        return pd.DataFrame(), pd.DataFrame(), None, "", None

    # Get file path from the uploaded file object
    file_path = file_obj.name if hasattr(file_obj, "name") else file_obj["name"]
    ext = os.path.splitext(file_path)[1].lower()

    # Initialize Spark session
    spark = SparkSession.builder.master("local[*]").appName("InconsistencyDetection").getOrCreate()

    try:
        if ext == ".csv":
            spark_df = spark.read.option("header", "true").csv(file_path)
        elif ext == ".txt":
            spark_df = spark.read.text(file_path)
            spark_df = spark_df.withColumnRenamed("value", "text_column")
        elif ext == ".json":
            try:
                spark_df = spark.read.json(file_path)
                if "text_column" not in spark_df.columns:
                    with open(file_path, 'r', encoding='utf-8') as f:
                        data = json.load(f)
                    if isinstance(data, list) and all(isinstance(x, str) for x in data):
                        spark_df = spark.createDataFrame([(x,) for x in data], ["text_column"])
                    else:
                        return pd.DataFrame({"Error": ["JSON file does not contain a 'text_column'."]}), pd.DataFrame(), None, "", None
            except Exception as e:
                return pd.DataFrame({"Error": [f"Error reading JSON file: {e}"]}), pd.DataFrame(), None, "", None
        elif ext == ".xml":
            try:
                df = pd.read_xml(file_path)
                if 'text_column' not in df.columns:
                    return pd.DataFrame({"Error": ["XML file does not contain a 'text_column'."]}), pd.DataFrame(), None, "", None
                spark_df = spark.createDataFrame(df)
            except Exception as e:
                return pd.DataFrame({"Error": [f"Error reading XML file: {e}"]}), pd.DataFrame(), None, "", None
        else:
            return pd.DataFrame({"Error": ["Unsupported file format. Supported formats: CSV, TXT, JSON, XML."]}), pd.DataFrame(), None, "", None
    except Exception as e:
        return pd.DataFrame({"Error": [f"Error reading file: {e}"]}), pd.DataFrame(), None, "", None

    # Define Spark UDFs wrapping our Python functions
    detect_formatting_udf = udf(lambda x: detect_formatting_issues(x), ArrayType(StringType()))
    detect_conflict_udf = udf(lambda x: detect_conflicting_information(x), ArrayType(StringType()))
    detect_anomalies_udf = udf(lambda x: detect_data_anomalies(x), ArrayType(StringType()))
    correct_text_udf = udf(lambda x: correct_text(x), StringType())

    # Apply UDFs to create new columns
    spark_df = spark_df.withColumn("formatting_issues", detect_formatting_udf(spark_df["text_column"]))
    spark_df = spark_df.withColumn("conflicting_information", detect_conflict_udf(spark_df["text_column"]))
    spark_df = spark_df.withColumn("data_anomalies", detect_anomalies_udf(spark_df["text_column"]))
    spark_df = spark_df.withColumn("corrected_text", correct_text_udf(spark_df["text_column"]))

    # Convert the Spark DataFrame to a Pandas DataFrame for further processing
    pdf = spark_df.toPandas()

    # Duplicate detection using pandas function
    duplicates_df = detect_duplicates(pdf)
    if duplicates_df.empty:
        duplicate_report = pd.DataFrame({"Message": ["No duplicate entries found."]})
    else:
        duplicate_report = duplicates_df.copy()

    processed_preview = pdf[['text_column', 'formatting_issues',
                             'conflicting_information', 'data_anomalies',
                             'corrected_text']]

    # Save the corrected DataFrame as a downloadable CSV file
    corrected_filename = "corrected_document.csv"
    pdf.to_csv(corrected_filename, index=False)

    # -------------------------------
    # Hadoop: Upload Corrected File to HDFS (optional)
    # -------------------------------
    if ENABLE_HADOOP:
        hdfs_destination = "/user/your_username/corrected_document.csv"  # Adjust based on your Hadoop config
        hadoop_status = upload_to_hdfs(corrected_filename, hdfs_destination)
    else:
        hadoop_status = "Hadoop integration disabled."

    # -------------------------------
    # Analysis & Graph Generation
    # -------------------------------
    total_records = len(pdf)
    records_with_errors = pdf.apply(lambda row: len(row['formatting_issues'] + row['conflicting_information'] + row['data_anomalies']) > 0, axis=1).sum()
    issues_counts = {}
    for idx, row in pdf.iterrows():
        issues = row['formatting_issues'] + row['conflicting_information'] + row['data_anomalies']
        for issue in issues:
            issues_counts[issue] = issues_counts.get(issue, 0) + 1

    analysis_text = (
        f"Analysis Summary:\n"
        f"Total records: {total_records}\n"
        f"Records with at least one inconsistency: {records_with_errors}\n"
        f"Breakdown of inconsistency types:\n"
    )
    for issue, count in issues_counts.items():
        analysis_text += f" - {issue}: {count} occurrence(s)\n"
    analysis_text += f"\nHadoop Upload Status: {hadoop_status}"

    fig, ax = plt.subplots(figsize=(8, 4))
    if issues_counts:
        x_vals = list(issues_counts.keys())
        y_vals = list(issues_counts.values())
        ax.plot(x_vals, y_vals, marker='o', linestyle='-')
        ax.set_title("Trend of Detected Inconsistencies")
        ax.set_xlabel("Inconsistency Type")
        ax.set_ylabel("Count")
        plt.xticks(rotation=45, ha="right")
    else:
        ax.text(0.5, 0.5, 'No inconsistencies detected', horizontalalignment='center',
                verticalalignment='center', transform=ax.transAxes, fontsize=12)
        ax.set_axis_off()
    plt.tight_layout()

    return processed_preview, duplicate_report, corrected_filename, analysis_text, fig

# -------------------------------
# Custom CSS for Black Background Interface
# -------------------------------

custom_css = """
body {
  background-color: #000;
  color: #fff;
}
.container {
  border: 1px solid #444;
  border-radius: 10px;
  padding: 20px;
  background-color: #222;
  margin: 10px;
}
h1, h2, h3, h4, h5, h6 {
  color: #fff;
  text-align: center;
}
"""

# -------------------------------
# Build and Launch the Gradio Blocks Interface
# -------------------------------

with gr.Blocks(css=custom_css) as demo:
    gr.Markdown("<h1>CSV Inconsistency Detection & Correction with Spark & Optional Hadoop</h1>")
    gr.Markdown(
        "Upload a file containing a column named **'text_column'**. "
        "This tool uses Apache Spark for data processing and (optionally) Hadoop for file storage. "
        "It detects formatting issues, conflicting information, and data anomalies, "
        "and then produces a corrected file for download. "
        "Supported formats: CSV, TXT, JSON, XML."
    )
    with gr.Row():
        with gr.Column(elem_classes="container"):
            file_input = gr.File(label="Upload File", file_count="single", file_types=['.csv', '.txt', '.json', '.xml'])
            process_button = gr.Button("Process Document")
        with gr.Column(elem_classes="container"):
            download_csv = gr.File(label="Download Corrected CSV")
    with gr.Row():
        with gr.Column(elem_classes="container"):
            processed_preview = gr.Dataframe(label="Processed Data Preview")
        with gr.Column(elem_classes="container"):
            duplicate_report = gr.Dataframe(label="Duplicate Report")
    with gr.Row():
        with gr.Column(elem_classes="container"):
            analysis_summary = gr.Textbox(label="Analysis Summary", lines=8)
        with gr.Column(elem_classes="container"):
            plot_output = gr.Plot(label="Inconsistencies Trend Graph")
    process_button.click(
        fn=process_file,
        inputs=file_input,
        outputs=[processed_preview, duplicate_report, download_csv, analysis_summary, plot_output]
    )

# Launch with debug=True for Colab to display errors and logs.
demo.launch(share=True, debug=True)


Colab notebook detected. This cell will run indefinitely so that you can see errors and logs. To turn off, set debug=False in launch().
* Running on public URL: https://a7806c9e02017c9818.gradio.live

This share link expires in 72 hours. For free permanent hosting and GPU upgrades, run `gradio deploy` from the terminal in the working directory to deploy to Hugging Face Spaces (https://huggingface.co/spaces)


Traceback (most recent call last):
  File "/usr/local/lib/python3.11/dist-packages/gradio/queueing.py", line 625, in process_events
    response = await route_utils.call_process_api(
               ^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^
  File "/usr/local/lib/python3.11/dist-packages/gradio/route_utils.py", line 322, in call_process_api
    output = await app.get_blocks().process_api(
             ^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^
  File "/usr/local/lib/python3.11/dist-packages/gradio/blocks.py", line 2103, in process_api
    result = await self.call_function(
             ^^^^^^^^^^^^^^^^^^^^^^^^^
  File "/usr/local/lib/python3.11/dist-packages/gradio/blocks.py", line 1650, in call_function
    prediction = await anyio.to_thread.run_sync(  # type: ignore
                 ^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^
  File "/usr/local/lib/python3.11/dist-packages/anyio/to_thread.py", line 56, in run_sync
    return await get_async_backend().run_sync_in_worker_thread(
           ^^^^^

In [ ]:
import os
import json
import time
import pandas as pd
import re
import gradio as gr
import matplotlib.pyplot as plt
from pyspark.sql import SparkSession
from pyspark.sql.functions import udf
from pyspark.sql.types import ArrayType, StringType

# -------------------------------
# Inconsistency Detection Functions
# -------------------------------

def detect_formatting_issues(text):
    """
    Checks whether the text:
      - Starts with an uppercase letter.
      - Ends with proper punctuation (., !, or ?).
    Returns a list of formatting issues.
    """
    issues = []
    if not text:
        issues.append("Empty text")
        return issues
    if not text[0].isupper():
        issues.append("Does not start with an uppercase letter")
    if text[-1] not in ['.', '!', '?']:
        issues.append("Does not end with proper punctuation")
    return issues

def detect_conflicting_information(text):
    """
    Checks for pairs of contradictory keywords in the text.
    Returns a list of detected conflicts.
    """
    conflict_pairs = [
        ('increase', 'decrease'),
        ('up', 'down'),
        ('true', 'false'),
        ('positive', 'negative'),
        ('win', 'loss'),
        ('hot', 'cold')
    ]
    conflicts = []
    text_lower = text.lower()
    for w1, w2 in conflict_pairs:
        if w1 in text_lower and w2 in text_lower:
            conflicts.append(f"Conflict: {w1} vs {w2}")
    return conflicts

def detect_data_anomalies(text):
    """
    Detects data anomalies such as:
      - Empty text.
      - Text that is extremely short.
      - Excessive punctuation (three or more punctuation marks in a row).
      - A high ratio of non-alphanumeric to alphanumeric characters.
    Returns a list of anomaly issues.
    """
    anomalies = []
    if not text or len(text.strip()) == 0:
        anomalies.append("Empty text anomaly")
    if len(text) < 5:
        anomalies.append("Text too short")
    if re.search(r'[\!\?\.]{3,}', text):
        anomalies.append("Excessive punctuation")
    alpha_numeric = re.findall(r'[a-zA-Z0-9]', text)
    non_alphanumeric = re.findall(r'[^a-zA-Z0-9\s]', text)
    if alpha_numeric and (len(non_alphanumeric) / len(alpha_numeric)) > 0.5:
        anomalies.append("High non-alphanumeric ratio")
    return anomalies

def detect_duplicates(df, text_column='text_column'):
    """
    Returns a DataFrame with rows that have duplicate values in the specified text column.
    """
    duplicates = df[df.duplicated(subset=[text_column], keep=False)]
    return duplicates

def correct_text(text):
    """
    Corrects formatting issues:
      - Capitalizes the first character if necessary.
      - Ensures the text ends with a punctuation mark (adds a period if missing).
    Returns the corrected text.
    """
    if not text:
        return text
    if not text[0].isupper():
        text = text[0].upper() + text[1:]
    if text[-1] not in ['.', '!', '?']:
        text = text + '.'
    return text

# -------------------------------
# Hadoop Integration Function
# -------------------------------
def upload_to_hdfs(local_file, hdfs_path):
    """
    Uploads a local file to HDFS using a shell command.
    Requires Hadoop to be installed and HDFS running.
    """
    command = f"hdfs dfs -put -f {local_file} {hdfs_path}"
    os.system(command)
    return f"File uploaded to HDFS at: {hdfs_path}"

# -------------------------------
# Gradio Interface Processing Function with Spark & Hadoop
# -------------------------------

def process_file(file_obj):
    if file_obj is None:
        return pd.DataFrame(), pd.DataFrame(), None, "", None, None, None

    # Start execution timer
    start_time = time.time()

    ext = os.path.splitext(file_obj.name)[1].lower()

    # Initialize Spark session
    spark = SparkSession.builder.master("local[*]").appName("InconsistencyDetection").getOrCreate()

    try:
        if ext == ".csv":
            spark_df = spark.read.option("header", "true").csv(file_obj.name)
        elif ext in [".txt"]:
            spark_df = spark.read.text(file_obj.name)
            spark_df = spark_df.withColumnRenamed("value", "text_column")
        elif ext in [".json"]:
            try:
                spark_df = spark.read.json(file_obj.name)
                if "text_column" not in spark_df.columns:
                    with open(file_obj.name, 'r', encoding='utf-8') as f:
                        data = json.load(f)
                    if isinstance(data, list) and all(isinstance(x, str) for x in data):
                        spark_df = spark.createDataFrame([(x,) for x in data], ["text_column"])
                    else:
                        return pd.DataFrame({"Error": ["JSON file does not contain a 'text_column'."]}), pd.DataFrame(), None, "", None, None, None
            except Exception as e:
                return pd.DataFrame({"Error": [f"Error reading JSON file: {e}"]}), pd.DataFrame(), None, "", None, None, None
        elif ext in [".xml"]:
            try:
                df = pd.read_xml(file_obj.name)
                if 'text_column' not in df.columns:
                    return pd.DataFrame({"Error": ["XML file does not contain a 'text_column'."]}), pd.DataFrame(), None, "", None, None, None
                spark_df = spark.createDataFrame(df)
            except Exception as e:
                return pd.DataFrame({"Error": [f"Error reading XML file: {e}"]}), pd.DataFrame(), None, "", None, None, None
        else:
            return pd.DataFrame({"Error": ["Unsupported file format. Supported formats: CSV, TXT, JSON, XML."]}), pd.DataFrame(), None, "", None, None, None
    except Exception as e:
        return pd.DataFrame({"Error": [f"Error reading file: {e}"]}), pd.DataFrame(), None, "", None, None, None

    # Define Spark UDFs wrapping our Python functions
    detect_formatting_udf = udf(lambda x: detect_formatting_issues(x), ArrayType(StringType()))
    detect_conflict_udf = udf(lambda x: detect_conflicting_information(x), ArrayType(StringType()))
    detect_anomalies_udf = udf(lambda x: detect_data_anomalies(x), ArrayType(StringType()))
    correct_text_udf = udf(lambda x: correct_text(x), StringType())

    # Apply UDFs to create new columns
    spark_df = spark_df.withColumn("formatting_issues", detect_formatting_udf(spark_df["text_column"]))
    spark_df = spark_df.withColumn("conflicting_information", detect_conflict_udf(spark_df["text_column"]))
    spark_df = spark_df.withColumn("data_anomalies", detect_anomalies_udf(spark_df["text_column"]))
    spark_df = spark_df.withColumn("corrected_text", correct_text_udf(spark_df["text_column"]))

    # Convert the Spark DataFrame to a Pandas DataFrame for further processing
    pdf = spark_df.toPandas()

    # Duplicate detection using our pandas function
    duplicates_df = detect_duplicates(pdf)
    if duplicates_df.empty:
        duplicate_report = pd.DataFrame({"Message": ["No duplicate entries found."]})
    else:
        duplicate_report = duplicates_df.copy()

    processed_preview = pdf[['text_column', 'formatting_issues',
                             'conflicting_information', 'data_anomalies',
                             'corrected_text']]

    # Save the corrected DataFrame as a downloadable CSV file
    corrected_filename = "corrected_document.csv"
    pdf.to_csv(corrected_filename, index=False)

    # -------------------------------
    # Hadoop: Upload Corrected File to HDFS
    # -------------------------------
    hdfs_destination = "/user/your_username/corrected_document.csv"
    hadoop_status = upload_to_hdfs(corrected_filename, hdfs_destination)

    # -------------------------------
    # Analysis & Graph Generation
    # -------------------------------
    total_records = len(pdf)
    records_with_errors = pdf.apply(lambda row: len(row['formatting_issues'] + row['conflicting_information'] + row['data_anomalies']) > 0, axis=1).sum()

    # Calculate accuracy rate as the percentage of records without issues
    accuracy_rate = ((total_records - records_with_errors) / total_records * 100) if total_records > 0 else 0

    issues_counts = {}
    for idx, row in pdf.iterrows():
        issues = row['formatting_issues'] + row['conflicting_information'] + row['data_anomalies']
        for issue in issues:
            issues_counts[issue] = issues_counts.get(issue, 0) + 1

    # End execution timer and compute processing time
    end_time = time.time()
    execution_time = end_time - start_time

    analysis_text = f"Analysis Summary:\nTotal records: {total_records}\n"
    analysis_text += f"Records with at least one inconsistency: {records_with_errors}\n"
    analysis_text += f"Accuracy Rate: {accuracy_rate:.2f}%\n"
    analysis_text += f"Execution Time: {execution_time:.2f} seconds\n"
    analysis_text += "Breakdown of inconsistency types:\n"
    for issue, count in issues_counts.items():
        analysis_text += f" - {issue}: {count} occurrence(s)\n"
    analysis_text += f"\nHadoop Upload Status: {hadoop_status}"

    # Simulate scale-up: Projected execution times for increasing dataset sizes.
    # (These factors are for demonstration purposes.)
    scale_factors = [1, 2, 3, 4, 5]
    simulated_times = [execution_time * factor * 0.9 for factor in scale_factors]

    # -------------------------------
    # Create Individual Graphs
    # -------------------------------

    # Graph 1: Accuracy Rate
    fig_accuracy, ax1 = plt.subplots(figsize=(6, 4))
    ax1.bar(["Accuracy Rate"], [accuracy_rate], color='green')
    ax1.set_ylim(0, 100)
    ax1.set_title("Accuracy Rate")
    ax1.set_ylabel("Percentage (%)")
    plt.tight_layout()

    # Graph 2: Execution Time
    fig_execution, ax2 = plt.subplots(figsize=(6, 4))
    ax2.bar(["Execution Time"], [execution_time], color='blue')
    ax2.set_title("Execution Time")
    ax2.set_ylabel("Time (seconds)")
    plt.tight_layout()

    # Graph 3: Scale-Up Analysis
    fig_scaleup, ax3 = plt.subplots(figsize=(6, 4))
    ax3.plot(scale_factors, simulated_times, marker='o', linestyle='-', color='orange')
    ax3.set_title("Scale-Up Analysis")
    ax3.set_xlabel("Scale Factor (Dataset Size Multiplier)")
    ax3.set_ylabel("Simulated Execution Time (s)")
    ax3.grid(True)
    plt.tight_layout()

    return processed_preview, duplicate_report, corrected_filename, analysis_text, fig_accuracy, fig_execution, fig_scaleup

# -------------------------------
# Custom CSS for Black Background Interface
# -------------------------------
custom_css = """
body {
  background-color: #000;
  color: #fff;
}
.container {
  border: 1px solid #444;
  border-radius: 10px;
  padding: 20px;
  background-color: #222;
  margin: 10px;
}
h1, h2, h3, h4, h5, h6 {
  color: #fff;
  text-align: center;
}
"""

# -------------------------------
# Build and Launch the Gradio Blocks Interface
# -------------------------------
with gr.Blocks(css=custom_css) as demo:
    gr.Markdown("<h1>CSV Inconsistency Detection & Correction with Spark & Hadoop</h1>")
    gr.Markdown(
        "Upload a file containing a column named **'text_column'**. "
        "This tool uses Apache Spark for data processing and Hadoop for file storage. "
        "It detects formatting issues, conflicting information, and data anomalies, "
        "and then produces a corrected file for download. "
        "Supported formats: CSV, TXT, JSON, XML."
    )

    with gr.Row():
        with gr.Column(elem_classes="container"):
            file_input = gr.File(label="Upload File", file_count="single", file_types=['.csv', '.txt', '.json', '.xml'])
            process_button = gr.Button("Process Document")
        with gr.Column(elem_classes="container"):
            download_csv = gr.File(label="Download Corrected CSV")

    with gr.Row():
        with gr.Column(elem_classes="container"):
            processed_preview = gr.Dataframe(label="Processed Data Preview")
        with gr.Column(elem_classes="container"):
            duplicate_report = gr.Dataframe(label="Duplicate Report")

    with gr.Row():
        with gr.Column(elem_classes="container"):
            analysis_summary = gr.Textbox(label="Analysis Summary", lines=8)

    with gr.Row():
        with gr.Column(elem_classes="container"):
            plot_accuracy = gr.Plot(label="Accuracy Rate")
        with gr.Column(elem_classes="container"):
            plot_execution = gr.Plot(label="Execution Time")
        with gr.Column(elem_classes="container"):
            plot_scaleup = gr.Plot(label="Scale-Up Analysis")

    process_button.click(
        fn=process_file,
        inputs=file_input,
        outputs=[processed_preview, duplicate_report, download_csv, analysis_summary,
                 plot_accuracy, plot_execution, plot_scaleup]
    )

demo.launch(share=True, inline=True)


Colab notebook detected. To show errors in colab notebook, set debug=True in launch()
* Running on public URL: https://0659dfbd1a4e19235d.gradio.live

This share link expires in 72 hours. For free permanent hosting and GPU upgrades, run `gradio deploy` from the terminal in the working directory to deploy to Hugging Face Spaces (https://huggingface.co/spaces)
